In [1]:
import requests
import pandas as pd
import random
from datetime import datetime
from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from amazoncaptcha import AmazonCaptcha
import time
import re
import os

In [2]:
# Definir la ruta de la carpeta y el archivo
folder_path = "../output"
csv_file = "data.csv"
csv_path = os.path.join(folder_path, csv_file)

# Verificar si la carpeta existe, si no, crearla
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Carpeta creada: {folder_path}")

# Si el archivo CSV no existe, crear un DataFrame vacío con las columnas adecuadas
if not os.path.exists(csv_path):
    data = pd.DataFrame(columns=["Product_Code", "Title", "Vendor_Name", "Rating", "Num_Ratings", "Price", "Quantity", "Min_Price_Comp", "Max_Price_Comp", "Avg_Price_Comp", "Date"])
    data.to_csv(csv_path, sep='|', index=False) 
else:
    data = pd.read_csv(csv_path, sep='|')

In [3]:
# Configurar las opciones de webdriver
options = Options()
# options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080") 

# Configurar el WebDriver Chrome
# driver_path = "../others/chromedriver"
# driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)

# Configurar el WebDriver Firefox
options.binary_location = 'C:\\Program Files\\Mozilla Firefox\\firefox.exe'
service = Service('../others/geckodriver.exe')
driver = webdriver.Firefox(service=service, options=options)

# Maximizar la ventana
driver.maximize_window()

# Abrir la página de Amazon
url = "https://www.amazon.com/" 
driver.get(url)

# Esperar unos segundos para que cargue la página
time.sleep(10)

try:
    # Localizar la única imagen y obtener su URL
    image = driver.find_element(By.TAG_NAME, "img")
    image_url = image.get_attribute("src")
    print("URL de la imagen:", image_url)

    # Descargar la imagen
    response = requests.get(image_url, stream=True)
    image_path = "../others/captured_image.jpg"
    with open(image_path, 'wb') as file:
        for chunk in response.iter_content(1024):
            file.write(chunk)
    print(f"Imagen descargada correctamente en: {image_path}")

    # Resolver el captcha
    captcha_value = AmazonCaptcha(image_path).solve()
    print(f"El texto del captcha es: {captcha_value}")

    # Ingresar cada carácter con un intervalo aleatorio
    captcha_input = driver.find_element(By.ID, "captchacharacters")
    for char in captcha_value:
        captcha_input.send_keys(char)
        time.sleep(random.uniform(0, 1))
    print("Captcha ingresado.")

    # Enviar el formulario del captcha
    captcha_input.send_keys(Keys.RETURN)
except:
    pass

print("Ingresaste a Amazon!")

URL de la imagen: https://images-na.ssl-images-amazon.com/captcha/sgkknrsj/Captcha_ekabjahapd.jpg
Imagen descargada correctamente en: ../others/captured_image.jpg
El texto del captcha es: KBEBCX
Captcha ingresado.
Ingresaste a Amazon!


In [4]:
try:
    # Cambiar localización para figurar en Miami
    location_link = driver.find_element(By.ID, "nav-global-location-popover-link")
    location_link.click()

    time.sleep(5)

    postal_code = "33101"
    postal_code_input = driver.find_element(By.XPATH, "//input[@autocomplete='postal-code']")
    for char in postal_code:
        postal_code_input.send_keys(char)
        time.sleep(random.uniform(0, 1))

    postal_code_button = driver.find_element(By.XPATH, "//span[@id='GLUXZipUpdate']")
    postal_code_button.click()

    print(f"Nos geolocalizamos en Miami.")
except:
    pass

In [5]:
product_id = 'B0B2RFPJY2'
product_url = url + f'gp/product/{product_id}'
driver.get(product_url)